# Transition Lookup creation

In [2]:
import numpy as np
from sklearn.utils.extmath import cartesian
from collections import namedtuple
import time
#from google.colab import files
import io
import pandas as pd
#from math import prod

# Probably inefficient implementation I need because colab is python 3.6
def prod(vec):
  res = 1
  for v in vec:
    res *= v
  return res

def create_market_states(max_firms):
  poss_firms = (range(n+1) for n in max_firms)
  return np.array(cartesian(poss_firms))

def create_all_transitions(market_states, max_firms):
  n_stages = len(max_firms)
  enter_transitions = [create_transition(market_states, max_firms, -1, i) for i in range(n_stages)]
  exit_transitions = [create_transition(market_states, max_firms, i, -1) for i in range(n_stages)]
  advance_from_transitions = [create_transition(market_states, max_firms, i, i+1) for i in range(n_stages-1)]
  # make them exit after last stage
  advance_from_transitions.append(create_transition(market_states, max_firms, n_stages-1, -1))
  Transitions = namedtuple('Transitions', ['enter','exit','advance_from'])
  return Transitions(enter_transitions, exit_transitions, advance_from_transitions)

# Creates a index lookup (for each state!) based on a firm leaving one stage and going to another.  stage == -1 means out of market.
# Assumes states are in canonical order. ie [[0,...,0,0], [0,...,0,1],...,[0,...,0,N], [0,...,1,0],...[N,...,N,N]]
# Sets index to 0 if the transition is infeasible
# Getting the correct index using max_firms=[N0, N1, N2, N3]. If from_stage=0 (first stage), the number of firms in that stage will be decreasing,
# so the new state will be "earlier". How many states behind is the state that has exactly 1 less firm in from_stage? It's (N1+1)*(N2+1)*(N3+1) behind (plus
# 1 is because there are zeros). Call this state state_tmp.
# If stage_to is 1, how many states is the new state "ahead" from the state_tmp? It's (N2+1)*(N3+1) ahead.
# Notice that the function does not distinguish between the impossible transition and the transition to the state [0,0,0,0] (in both cases will give 0)
# But as long as the value of the value function at [0, 0, 0, 0] is zero, that should be ok
def create_transition(market_states, max_firms, from_stage, to_stage):
  n_states = len(market_states)
  ix = np.arange(n_states)
  possible_transitions = np.ones(n_states, dtype=bool)
  if from_stage >= 0:
    possible_transitions *= market_states[:, from_stage] > 0
    ix -= prod(max_firms[from_stage + 1:] + 1)
  if to_stage >= 0:
    possible_transitions *= market_states[:, to_stage] < max_firms[to_stage]
    ix += prod(max_firms[to_stage + 1:] + 1)
  return ix * possible_transitions

# Rates of state changes and corresponding lookups
The dynamic programming state variable is (my_stage, market_state).
This can change for any number of reasons, opponent enters/exits/advances, or I exit/advance.  These moves can be forced by the system or chosen by the firm.  Regardless, we need to find the expected value received by the firm in the future.  To do this, need to list all possible state transitions and their corresponding probability.

Transition rates are the lambdas of exponentials.  When we are drawing form many heterogeneous exponentials, the probability that random variable i is lowest (and hence the first state change to occur) is give by: $\lambda_i / \sum(\lambda_i)$.

Finally, since our goal is to iterate to find the value function, we'll need to take each possible transition, find the state it ends up at, the probability of that transition occuring, and then multiply that probability by the expected value of that new state.

To do this, most of the following functions return a (transition_lookup, transition_rate) pair.  Both the lookup and the rate are matrices, with one row for each possible state change (similar state changes grouped together).  The lookup gives the indices of the new market state, and the rates give the unnormalized probability of that state change occurring.

In [ ]:
# Transitions where this firm exits (no lookup matrix since terminal state)
def create_me_forced_exit(market_states, rates, transitions, checks, my_stage):
  # Must exit upon bad signal
  rate_bad = rates.stage_end[my_stage] * (1 - rates.good_signal[my_stage])
  # Must exit on good signal if this is the last stage or next stage is full
  rate_good_forced = rates.stage_end[my_stage] * rates.good_signal[my_stage] * checks.invalid_advance_from[:, my_stage]
  return LookupRate(lookups=None, rates=(rate_bad + rate_good_forced) * checks.are_firms[:,my_stage])

# Transitions where this firm is forced to go to the next stage (stage is exogenous, good signal, valid move).
#This will be non-empty only for stage 3 (indexed as 2 in python)
def create_me_forced_advance(market_states, rates, transitions, checks, my_stage):
  lookups = transitions.advance_from[my_stage].copy()
  trans_rates = rates.stage_end[my_stage] * rates.good_signal[my_stage] * checks.valid_advance_from[:, my_stage] * checks.is_exogenous[:,my_stage]
  return filter_transition_rates(lookups[None, :], trans_rates[None,:], checks, my_stage)

# Transitions where this firm gets to decide on a course of action (stage endogenous, good signal, can move ahead)
def create_me_choose(market_states, rates, transitions, checks, my_stage):
  trans_rates = rates.stage_end[my_stage] * rates.good_signal[my_stage] * checks.valid_advance_from[:, my_stage] * checks.is_endogenous[:,my_stage]
  return LookupRate(lookups=None, rates=trans_rates * checks.are_firms[:,my_stage])

# Gets number of competitors is each market_state (subtracts 1 from my stage from each state, assuming there's at least 1 firm.)
def get_other_states(market_states, checks, my_stage):
  other_states = market_states.copy()
  other_states[:, my_stage] -= checks.are_firms[:,my_stage]
  return other_states

# Transitions where another firm is forced to change states
def create_other_forced(market_states, rates, transitions, checks, my_stage):
  n_stages = len(market_states[0])
  other_states = get_other_states(market_states, checks, my_stage)
  lookups = []
  trans_rates = []
  for i in range(n_stages):
    lookups.append(transitions.enter[i].copy())
    trans_rates.append(rates.enter[i] * checks.valid_advance_to[:,i])
  for i in range(n_stages):
    lookups.append(transitions.exit[i].copy())
    trans_rates.append(rates.stage_end[i] * other_states[:,i] * (1 - rates.good_signal[i] * checks.valid_advance_from[:,i]))
  for i in range(n_stages):
    lookups.append(transitions.advance_from[i].copy())
    trans_rates.append(rates.stage_end[i] * other_states[:,i] * rates.good_signal[i] * checks.valid_advance_from[:,i] * checks.is_exogenous[:,i])
  lookups = np.stack(lookups)
  trans_rates = np.stack(trans_rates)
  return filter_transition_rates(lookups, trans_rates, checks, my_stage)

# Transitions where another firm is endogenously chooses to advance a stage
def create_other_advance(market_states, rates, transitions, checks, my_stage):
  n_stages = len(market_states[0])
  other_states = get_other_states(market_states, checks, my_stage)
  lookups = []
  trans_rates = []
  for i in range(n_stages):
    lookups.append(transitions.advance_from[i])
    # Note rates will be modulated by the choice probabilities later in the code.
    trans_rates.append(rates.stage_end[i] * other_states[:, i] * rates.good_signal[i] * checks.valid_advance_from[:,i] * checks.is_endogenous[:,i])
  lookups = np.stack(lookups)
  trans_rates = np.stack(trans_rates)
  return filter_transition_rates(lookups, trans_rates, checks, my_stage)

# Transitions where another firm endogenously chooses to exit the market
def create_other_exit(market_states, rates, transitions, checks, my_stage):
  n_stages = len(market_states[0])
  other_states = get_other_states(market_states, checks, my_stage)
  lookups = []
  trans_rates = []
  for i in range(n_stages):
    lookups.append(transitions.exit[i])
    # Note rates will be modulated by the choice probabilities later in the code.
    trans_rates.append(rates.stage_end[i] * other_states[:, i] * rates.good_signal[i] * checks.valid_advance_from[:,i] * checks.is_endogenous[:,i])
  lookups = np.stack(lookups)
  trans_rates = np.stack(trans_rates)
  return filter_transition_rates(lookups, trans_rates, checks, my_stage)

# Helper function for eliminating rows with 0 probability of occurring.
LookupRate = namedtuple('LookupRate', ['lookups', 'rates'])
def filter_transition_rates(lookups, rates, checks, my_stage):
  rates *= checks.are_firms[:, my_stage]
  lookups *= rates != 0
  total_rates = np.sum(rates, axis=1)
  return LookupRate(lookups[total_rates != 0].astype(np.int32), rates[total_rates != 0])

In [ ]:
# Calculate E(max(V + e(1), e(0)))
# Theory says that if e's are extreme value distributions, then Expectation = log(1 + exp(V)) + euler_gamma
# note that numpy on cpu overflows shortly above exp(700) (GPU even less).  But, log(1 + exp(x)) ~ x when x is large, so will use that approx when x above 700.
def expected_choice_value(advance_values, choice_stages, checks):
  max_exponent = 700
  will_overflow = advance_values > max_exponent
  capped_exp = np.minimum(advance_values, max_exponent)
  expect_max = will_overflow * (advance_values + np.euler_gamma)
  expect_max += (1 - will_overflow) * (np.log(1 + np.exp(capped_exp)) + np.euler_gamma) 
  expect_max *= checks.are_firms[:, choice_stages].T * checks.valid_advance_from[:, choice_stages].T
  return expect_max

def calculate_updated_values(values, flow_payoffs, choice_stages, expected_max, discounted_rates, actions, choose_advance_prob):
  n_stages = flow_payoffs.shape[1]
  other_advance_rates = [actions.other_advance[i].rates * choose_advance_prob for i in range(n_stages)]
  other_exit_rates = [actions.other_exit[i].rates * (1 - choose_advance_prob) for i in range(n_stages)]
  value_updates = []
  for i in range(n_stages):
    # Expect to be in state for 1/discounted_rates time
    this_state_payoff = flow_payoffs[:,i] / discounted_rates[i]
    future_val = (actions.me_forced_exit[i].rates * np.euler_gamma +
                  np.sum(actions.other_forced[i].rates * values[i][actions.other_forced[i].lookups], 0) +
                  np.sum(other_advance_rates[i] * values[i][actions.other_advance[i].lookups], 0) +
                  np.sum(other_exit_rates[i] * values[i][actions.other_exit[i].lookups], 0))
    if i in choice_stages:
      # TODO: This breaks if endogenous stages aren't the first n stages 
      future_val += actions.me_choose[i].rates * expected_max[i,:]
    if i != n_stages-1:
      future_val += np.sum(actions.me_forced_advance[i].rates * (values[i+1][actions.me_forced_advance[i].lookups]), 0)
    # Normalize the rates into probabilities
    future_val /= discounted_rates[i]
    value_updates.append(this_state_payoff + future_val)
  return np.stack(value_updates)

In [ ]:
#Uploading the parameter file
#uploaded = files.upload()

#Mounting the google drive instead:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Importing the parameter file
#df_params = pd.read_csv(io.BytesIO(uploaded['params_for_counterfactual_all.csv']))

#Loading params from the drive:
params_file = 'params_for_counterfactual_all_3.csv'
path = F"/content/gdrive/My Drive/DrugDevelopment_ModelSolution/params/{params_file}"
#model.load_state_dict(torch.load(path))
df_params = pd.read_csv(path)
df_params = df_params[df_params['batch']==5]
df_params = df_params.reset_index()
df_params

,index,Indication,indications_num,p_1,p_2,p_3,n_start_1,n_start_2,n_start_3,n_start_4,max_in_1,max_in_2,max_in_3,max_in_4,state_space_final,to_use,c_2,c_3,theta_0,theta_r,theta_s,stageduration_1_years,stageduration_2_years,stageduration_3_years,stageduration_4_years,years_till_generic,entryduration_1_years,entryduration_2_years,entryduration_3_years,entryduration_4_years,rho,marketsize,batch
0,22,Parkinsons disease,23,0.223967,0.239803,0.844828,33,5,2,3,247,48,12,26,4265352.0,1,-0.04981,-0.055226,2.401461,0.006612,-0.634954,8.232080,7.805088,2.308219,170.640455,20,0.057076,0.658048,9.089372,158.150685,0.077962,111.112638,5
1,26,Schizophrenia,27,0.221569,0.385153,0.844828,22,4,2,3,93,44,12,34,1924650.0,1,-0.04981,-0.055226,2.401461,0.006612,-0.634954,5.058969,6.240396,1.682192,170.640455,20,0.090750,0.548834,9.089372,158.150685,0.077962,236.818813,5


In [ ]:
# max_firms = np.array([0,0,0,0])
# max_firms[0]
# n_stages = 4

# index=0
# max_firms = np.array([0,0,0,0])
# for i in range(4):
#   name='max_in_'+str(i+1)
#   max_firms[i]=df_params.loc[index][name]

# max_firms

# df_params.loc[index]['c_2']
# df_params.loc[index]['indications_num']

In [ ]:
n_stages = 4


p_increase_1=0.01
p_increase_2=0

c_2_decrease_percent=0

tax_percent=0

is_choice_stage = np.array([True, True, False, False])
choice_stages = np.array([0,1])

start_time = time.time()

index=0
#for index in range(df_params.shape[0]):
max_firms = np.array([0,0,0,0])

indication_number=df_params.loc[index]['indications_num']

for i in range(4):
  name='max_in_'+str(i+1)
  max_firms[i]=df_params.loc[index][name]

flow_costs = np.array([0, 
                        (1-c_2_decrease_percent)*df_params.loc[index]['c_2'], 
                        df_params.loc[index]['c_3'], 
                        0])
jump_costs = np.array([0, 0])
theta_0 = (1-tax_percent)*np.array([0, 0, 0, df_params.loc[index]['theta_0']])
theta_r = (1-tax_percent)*np.array([0, 0, 0, df_params.loc[index]['theta_r']])
theta_s = (1-tax_percent)*np.array([0, 0, 0, df_params.loc[index]['theta_s']])

#Entry of generics is introduced through increaing the discount rate
#when the firm is on the market (as in the value function iteration they are equivalent
# and both enter only through the denominator that contains the sum of all the rates)
#Rate for generics = discount rate on the market - discount rate at other stages
disc_rate_4=df_params.loc[index]['rho']+1/df_params.loc[index]['years_till_generic']
discount_rates = np.array([df_params.loc[index]['rho'], df_params.loc[index]['rho'], df_params.loc[index]['rho'], disc_rate_4])

market_states = create_market_states(max_firms)
n_states = len(market_states)

at_max_firms = market_states >= max_firms
# If the next stage is full, firms who might otherwise advance are forced to exit
invalid_advance_from = np.append(at_max_firms[:,1:], np.ones((n_states, 1), dtype=bool), axis=1)
Checks = namedtuple("Checks", ['are_firms', 'valid_advance_to', 'valid_advance_from', 'invalid_advance_from', 'is_endogenous', 'is_exogenous'])
checks = Checks(market_states > 0,
                market_states < max_firms,
                (1-invalid_advance_from).astype(bool),
                invalid_advance_from,
                np.ones_like(market_states, dtype=bool) * is_choice_stage,
                np.ones_like(market_states, dtype=bool) * (1-is_choice_stage).astype(bool))

transitions = create_all_transitions(market_states, max_firms)

market_size = df_params.loc[index]['marketsize']

Rates = namedtuple("Rates", ['stage_end', 'enter', 'good_signal'])
#When solving the original model, I will include the exogenous entry for stages 1, 2, 3 
rates = Rates(np.array([1/df_params.loc[index]['stageduration_1_years'], 
                        1/df_params.loc[index]['stageduration_2_years'], 
                        1/df_params.loc[index]['stageduration_3_years'], 
                        1/df_params.loc[index]['stageduration_4_years']]),
              np.array([1/df_params.loc[index]['entryduration_1_years'], 
                        1/df_params.loc[index]['entryduration_2_years'], 
                        1/df_params.loc[index]['entryduration_3_years'], 
                        0]),
              np.array([df_params.loc[index]['p_1']+p_increase_1, 
                        df_params.loc[index]['p_2']+p_increase_2, 
                        df_params.loc[index]['p_3'], 
                        0]))

Actions = namedtuple("Actions", ['me_forced_exit', 'me_forced_advance', 'me_choose', 'other_forced', 'other_advance', 'other_exit'])
actions = Actions(me_forced_exit = [create_me_forced_exit(market_states, rates, transitions, checks, i) for i in range(n_stages)],
                  me_forced_advance = [create_me_forced_advance(market_states, rates, transitions, checks, i)for i in range(n_stages)],
                  me_choose = [create_me_choose(market_states, rates, transitions, checks, i)for i in range(n_stages)],
                  other_forced = [create_other_forced(market_states, rates, transitions, checks, i)for i in range(n_stages)],
                  other_advance = [create_other_advance(market_states, rates, transitions, checks, i)for i in range(n_stages)],
                  other_exit = [create_other_exit(market_states, rates, transitions, checks, i)for i in range(n_stages)])

rate_state_change = np.sum(rates.enter * checks.valid_advance_to, axis=1) + np.sum(rates.stage_end * market_states, axis=1)
discounted_rates = [rate_state_change + discount_rate for discount_rate in discount_rates]
flow_payoffs = checks.are_firms * (flow_costs + theta_0 +
                                  theta_r * np.log(market_size) +
                                  theta_s * np.log(1 + market_states))

# values = np.ones((n_stages, n_states)) * 100 * checks.are_firms.T
# choose_advance_prob_old = np.zeros((len(choice_stages), n_states))
# tol_vf = 0.01
# tol_ccp = 0.00001
# it = 0
# max_it = 10000
# norm_vf = 1
# norm_ccp = 1
# learning_rate = 1
# while (it < max_it) and (norm_vf > tol_vf or norm_ccp > tol_ccp):
#   # Expected value from advancing (ignoring error)
#   advance_values = np.array([jump_costs[i] + values[i+1, transitions.advance_from[i]] for i in choice_stages] )
#   # Probability this firm would choose to advance (accounting for error from extreme value dist)
#   choose_advance_prob = np.stack([checks.are_firms[:,choice_stages[i]] * 1 / (1 + np.exp(-advance_values[i])) for i in range(len(choice_stages))])
#   expected_max = expected_choice_value(advance_values, choice_stages, checks)
#   updated_values = calculate_updated_values(values, flow_payoffs, choice_stages, expected_max, discounted_rates, actions, choose_advance_prob)

#   norm_ccp = np.max(np.abs(choose_advance_prob_old - choose_advance_prob))
#   norm_vf = np.max(np.abs(values - updated_values))
#   choose_advance_prob_old = choose_advance_prob
#   values = values + learning_rate * (updated_values - values)
#   it += 1
# print(index, it)
# values=np.transpose(values)
# choose_advance_prob_old=np.transpose(choose_advance_prob_old)
#filename_1='ctps_p1plus1_indication_'+str(indication_number)+'_2.txt'
#path_1 = F"/content/gdrive/My Drive/DrugDevelopment_ModelSolution/{filename_1}"
#np.savetxt(path_1, choose_advance_prob_old, delimiter='\t')
#filename_2='vs_p1plus1_indication_'+str(indication_number)+'_2.txt'
#path_2 = F"/content/gdrive/My Drive/DrugDevelopment_ModelSolution/{filename_2}"
#np.savetxt(path_2, values, delimiter='\t')
#files.download(filename_2)
#print(sign, rep, it)
# elapsed_time = time.time() - start_time



In [ ]:
theta_r*np.log(market_size)

array([0.        , 0.        , 0.        , 0.03114541])

In [ ]:
theta_r

array([0.        , 0.        , 0.        , 0.00661185])

In [ ]:
theta_r*market_size

array([0.        , 0.        , 0.        , 0.73466009])

In [ ]:
# for index in range(2):
#   indication_number=df_params.loc[index]['indications_num']
#   files.download('ctps_p2plus10pp_indication_'+str(indication_number)+'.txt')
#   files.download('vs_p2plus10pp_indication_'+str(indication_number)+'.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
list(range(1,df_params.shape[0]))

[1]

In [ ]:
list(range(2,4))

[2, 3]